In [62]:
import spacy
import gzip
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from langdetect import detect
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import decomposition
from stop_words import get_stop_words
from scipy.spatial import distance

In [3]:
def read_file(f):
    with gzip.open(f, 'rt', encoding='utf-8') as inf:
        j = json.load(inf)
        return j[0]['CallZText'], j[0]['CallZType']
read_file('1551.gov.ua/raw/А/А-1025.json.gz')

('Я, як мешканець Оболонського району занепокоєний ситуацією, яка склалася з незадовільним станом освітлення під’їзду №2 будинку за адресою вулиця Героїв Дніпра 38. Вимагаю від Вас терміново звернути увагу на цю проблему та вирішити її (перевірити стан проводки, закрити доступ до неї плафоном). \r\nДля підтвердження усунення недопрацювання прошу виконавця надіслати фотопідтвердження!',
 'Ремонт та заміна електроприладів')

In [4]:
data = []
import glob
for f in glob.glob('1551.gov.ua/raw/*/*.gz'):
    data.append(read_file(f))
data[0]

('Проблема: у нашому  мікрорайоні немає контейнерів для роздільного збору відходів. \r\nНавіть немає куди викинути пачку газет або скляну банку. Всі ми прагнемо жити по-європейськи, але ж у Європі люди давно викидають сміття у різні контейнери!\r\nПропозиціЇ:  1. КП ”Нивки” укласти договір з будь-якою організацією, що займається вивезенням, сортуванням та переробкою відходів. Наприклад, ТОВ ”Грінко-Київ”.\r\n2. Встановити контейнери євростандарт об”ємом 1,1 куб.м., або типу ”дзвін” для роздільного збору відходів: пластмаси, скла, макулатури та металу. Необхідно щонайменше 1 такий контейнер біля будинку по вул. Полкова, 55, та 3 таких  контейнери на майданчику для відходів, який розташований між будинками по вул.Полкова, 57 Б та 57В, вул.Межова, 119 та пров. Полковий,1.',
 'Встановлення сміттєвих контейнерів та урн для сміття')

In [5]:
len(data)

127328

In [6]:
def langdetect(text):
    try:
        language = detect(text)
    except:
        language = "error"
    return language

In [7]:
df = pd.DataFrame(data=data, columns=['text', 'cat'])
df['lang'] = df['text'].apply(lambda x : langdetect(x))
df = df[df['lang']=='uk']
df.head()

,text,cat,lang
0,"Проблема: у нашому мікрорайоні немає контейнерів для роздільного збору відходів. \r\nНавіть немає куди викинути пачку газет або скляну банку. Всі ми прагнемо жити по-європейськи, але ж у Європі люди давно викидають сміття у різні контейнери!\r\nПропозиціЇ: 1. КП ”Нивки” укласти договір з будь-якою організацією, що займається вивезенням, сортуванням та переробкою відходів. Наприклад, ТОВ ”Грінко-Київ”.\r\n2. Встановити контейнери євростандарт об”ємом 1,1 куб.м., або типу ”дзвін” для роздільного збору відходів: пластмаси, скла, макулатури та металу. Необхідно щонайменше 1 такий контейнер біля будинку по вул. Полкова, 55, та 3 таких контейнери на майданчику для відходів, який розташований між будинками по вул.Полкова, 57 Б та 57В, вул.Межова, 119 та пров. Полковий,1.",Встановлення сміттєвих контейнерів та урн для сміття,uk
1,"Доброго вечора!\r\nЗа адресою Леся Курбаса 18 в 5 під*їзді не працює ліфт з 20-00 години, та телефон диспетчерської частини 424-04-06 не відповідає!\r\nДопоможіть з вирішенням проблеми!Адже тут проживають і люди похолого віку, і вагітні жінки, та новонароджені дітки.\r\n\r\nІ вкажіть будь-ласка номер телефону який доступний цілодобово.",Не працює пасажирський ліфт,uk
2,"Добрий день! \r\n Моя скарга полягає в тому, що по вказаній адресі вже на протязі тижня у підвалі будинку знаходиться вода, звідки вона взялась - невідомо. В результаті чого, розмокають стіни і стеля, складаються антисанітарні умови. \r\n Наш будинок обслуговує ЖЕК - 0318, на рахунок якого ми, мешканці, сплачуємо за утримання будинку та прибудинкової території. Сьогодні зранку, 19.05.2014р., прийшов сантехник із цього ЖЕК, покружляв по підвалу, щось подивився, сказав, що ні чого не знайшов і пішов. \r\n Що треба зробити, щоб за наші кошти нам надавили якісні послуги та інформацію?! \r\n Мене цікавить: причина появи води, коли її викачають і хто повинен цим займатись?!\r\n\r\nБудь-ласка, розберіться з цим питанням!\r\n\r\nЗ повагою, Зінькова Тетяна\r\n19.05.2014р.",Порив труб у підвальному приміщенні,uk
3,Прошу звернути увагу на незаконну діяльність СТО на території спортивного комплексу біля будинку 31 по вул. Максима Берлинського: постійний шум з ранку до вечора навіть у вихідних!,Будівництво АЗС,uk
4,"Хочу поскаржитись особисто Береговому Петру Івановичу на те, що у його відповіді на моє звернення З-5976 від 20/08/2013, яке стосується звернення З-5300 від 25/07/2013, надано невірну інформацію.\r\n\r\nУ зверненні З-5976 йде мова про ями на Голосіївському проспекті, а у відповіді - про решітки на вул Заболотного. Хтось щось у вас наплутав.\r\n\r\nТим часом ями все ще не відремонтовано. Сьогодні 28/08/2013. Пішов другий місяць.",Технічний стан проїжджих частин вулиць та тротуарів,uk


In [8]:
uk = spacy.load("/tmp/la_vectors_wiki_lg")
stop_words = get_stop_words('uk')

def vec(word):
    return uk(word)[0].vector

def doc_vector(text):
    words = [token.text for token in uk(text) if token.has_vector if token.text not in stop_words and not token.text.isdigit()]
    w = ' '.join(words)
    vector = np.mean([vec(w) for w in words], axis=0)
    return w, len(words), vector

In [9]:
%%time
res = df.apply(lambda row: doc_vector(row['text']),axis=1)

/home/liudmyla/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/liudmyla/.local/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 1min 10s, sys: 127 ms, total: 1min 10s
Wall time: 1min 10s


In [10]:
df['words'], df['size'], df['vect'] =  [x for (x,y,z) in res], [y for (x,y,z) in res], [z for (x,y,z) in res]

In [11]:
df.head(1)

,text,cat,lang,words,size,vect
0,"Проблема: у нашому мікрорайоні немає контейнерів для роздільного збору відходів. \r\nНавіть немає куди викинути пачку газет або скляну банку. Всі ми прагнемо жити по-європейськи, але ж у Європі люди давно викидають сміття у різні контейнери!\r\nПропозиціЇ: 1. КП ”Нивки” укласти договір з будь-якою організацією, що займається вивезенням, сортуванням та переробкою відходів. Наприклад, ТОВ ”Грінко-Київ”.\r\n2. Встановити контейнери євростандарт об”ємом 1,1 куб.м., або типу ”дзвін” для роздільного збору відходів: пластмаси, скла, макулатури та металу. Необхідно щонайменше 1 такий контейнер біля будинку по вул. Полкова, 55, та 3 таких контейнери на майданчику для відходів, який розташований між будинками по вул.Полкова, 57 Б та 57В, вул.Межова, 119 та пров. Полковий,1.",Встановлення сміттєвих контейнерів та урн для сміття,uk,"Проблема нашому мікрорайоні контейнерів роздільного збору відходів Навіть викинути пачку газет скляну банку Всі прагнемо жити європейськи Європі викидають сміття різні контейнери КП Нивки укласти договір якою організацією займається вивезенням переробкою відходів Наприклад ТОВ Грінко Київ Встановити контейнери 1,1 типу дзвін роздільного збору відходів пластмаси скла макулатури металу Необхідно щонайменше контейнер біля будинку вул. таких контейнери майданчику відходів розташований будинками вул Б вул Межова пров.",65,"[-0.32706705, 0.35372034, -0.8181365, -0.09763757, -0.18483746, 0.13988332, -0.17825137, 0.17031503, -0.38090488, 0.07560193, -0.24593158, 0.5001311, 0.6368565, -0.56226766, 0.1064828, -0.6444452, 0.5123788, 0.7801814, -0.11225351, -0.0143884495, 0.7811906, 0.7186587, 0.6559944, 0.2557409, 0.26502132, 0.052653603, -0.030327711, 0.1281511, 0.26041865, 0.018589986, -0.3749531, -0.027035872, 0.018411936, 0.20674707, -0.47262248, 0.25025842, -0.21917783, 0.3849633, 0.2484838, 0.2125767, 0.2647651, -0.17051622, 0.29916203, 0.46446344, 0.14572063, -0.13194916, 0.039971646, 0.5668594, 0.6774498, -0.5519976, 0.30110857, 0.17862453, -0.29349995, -0.17421632, 0.2637783, 0.27411562, -0.14921941, -0.54453504, -0.12101258, -0.032360576, -0.14878331, -0.11509999, -0.1723311, 0.10172971, 0.7325146, -0.46720406, 0.68554497, -0.25038415, -0.2716254, 0.32725686, 0.36161917, -0.22139357, 0.19433327, -0.052507054, 0.24527335, 0.1926445, 0.10404079, -0.5297064, 0.10759066, -0.38590762, 0.030625908, -0...."


In [12]:
df = df[df['size'] > 1]
len(df)

68816

In [13]:
df.to_csv('vectors.csv', index=False)

In [39]:
freq = df.groupby('cat', as_index=False)['text'].count().sort_values('text', ascending=False)
f_cat = freq[freq['text']>10]['cat'].tolist()
len(f_cat)

397

In [40]:
df = df[df['cat'].isin(f_cat)]
len(df)

67200

In [41]:
cat_unique = list(set(df['cat'].tolist()))
values = [i for i in range(len(cat_unique))]
d = dict(zip(cat_unique,values))
df['target'] = df['cat'].apply(lambda x : d[x])

In [42]:
df.head(1)

,text,cat,lang,words,size,vect,target
0,"Проблема: у нашому мікрорайоні немає контейнерів для роздільного збору відходів. \r\nНавіть немає куди викинути пачку газет або скляну банку. Всі ми прагнемо жити по-європейськи, але ж у Європі люди давно викидають сміття у різні контейнери!\r\nПропозиціЇ: 1. КП ”Нивки” укласти договір з будь-якою організацією, що займається вивезенням, сортуванням та переробкою відходів. Наприклад, ТОВ ”Грінко-Київ”.\r\n2. Встановити контейнери євростандарт об”ємом 1,1 куб.м., або типу ”дзвін” для роздільного збору відходів: пластмаси, скла, макулатури та металу. Необхідно щонайменше 1 такий контейнер біля будинку по вул. Полкова, 55, та 3 таких контейнери на майданчику для відходів, який розташований між будинками по вул.Полкова, 57 Б та 57В, вул.Межова, 119 та пров. Полковий,1.",Встановлення сміттєвих контейнерів та урн для сміття,uk,"Проблема нашому мікрорайоні контейнерів роздільного збору відходів Навіть викинути пачку газет скляну банку Всі прагнемо жити європейськи Європі викидають сміття різні контейнери КП Нивки укласти договір якою організацією займається вивезенням переробкою відходів Наприклад ТОВ Грінко Київ Встановити контейнери 1,1 типу дзвін роздільного збору відходів пластмаси скла макулатури металу Необхідно щонайменше контейнер біля будинку вул. таких контейнери майданчику відходів розташований будинками вул Б вул Межова пров.",65,"[-0.32706705, 0.35372034, -0.8181365, -0.09763757, -0.18483746, 0.13988332, -0.17825137, 0.17031503, -0.38090488, 0.07560193, -0.24593158, 0.5001311, 0.6368565, -0.56226766, 0.1064828, -0.6444452, 0.5123788, 0.7801814, -0.11225351, -0.0143884495, 0.7811906, 0.7186587, 0.6559944, 0.2557409, 0.26502132, 0.052653603, -0.030327711, 0.1281511, 0.26041865, 0.018589986, -0.3749531, -0.027035872, 0.018411936, 0.20674707, -0.47262248, 0.25025842, -0.21917783, 0.3849633, 0.2484838, 0.2125767, 0.2647651, -0.17051622, 0.29916203, 0.46446344, 0.14572063, -0.13194916, 0.039971646, 0.5668594, 0.6774498, -0.5519976, 0.30110857, 0.17862453, -0.29349995, -0.17421632, 0.2637783, 0.27411562, -0.14921941, -0.54453504, -0.12101258, -0.032360576, -0.14878331, -0.11509999, -0.1723311, 0.10172971, 0.7325146, -0.46720406, 0.68554497, -0.25038415, -0.2716254, 0.32725686, 0.36161917, -0.22139357, 0.19433327, -0.052507054, 0.24527335, 0.1926445, 0.10404079, -0.5297064, 0.10759066, -0.38590762, 0.030625908, -0....",98


In [43]:
t = list(zip(df.cat, df.vect))
c = {}
for category in cat_unique:
    c[category] = np.mean([y for (x,y) in t if x==category], axis=0)

In [44]:
vectors = list(c.values())
len(vectors)

397

In [45]:
X = np.array(vectors)
X.shape

(397, 300)

In [46]:
%%time
from sklearn.cluster import AffinityPropagation
clustering = AffinityPropagation(random_state=42, verbose=True).fit(X)

Converged after 23 iterations.
CPU times: user 153 ms, sys: 76.2 ms, total: 230 ms
Wall time: 42.1 ms


In [88]:
labels = clustering.labels_.tolist()
res = tuple(zip(cat_unique,labels))
len(set(labels))

36

In [103]:
group = []
for i in range(36):
    group.append([x for (x,y) in res if y == i])
group[20]

['Впровадження нових автобусних зупинок',
 'Відновлення автобусних маршрутів',
 'Зміна траси існуючих автобусних маршрутів',
 'Організація нових трамвайних маршрутів',
 'Зміна траси існуючих тролейбусних маршрутів',
 'Перенесення трамвайних зупинок',
 'Інші питання, пропозиції щодо роботи міської електрички',
 'Облаштування діючих станцій міської електрички',
 'Пропозиції та запитання щодо удосконалення роботи громадського транспорту',
 'Перенесення автобусних зупинок',
 'Незадовільний технічний стан тролейбусів',
 'Пропозиції щодо вдосконалення роботи маршрутних таксі',
 'Організація нових автобусних маршрутів',
 'Ліквідація маршрутних таксі',
 'Зупинки на вимогу маршрутних таксі',
 'Розмір тарифів на проїзд у маршрутних таксі',
 'Переповнені тролейбуси (недостатня кількість)']

In [129]:
def category(text):
    for (x,y) in res:
        if x==text:
            return y 

In [132]:
df['group_cat'] = df.apply(lambda row: category(row['cat']),axis=1)

In [134]:
X_train, X_test, y_train, y_test = train_test_split(df['vect'].values, df['group_cat'].values, test_size=0.3, random_state=42, stratify=df['group_cat'].values, shuffle=True)

In [135]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=5)

In [136]:
KNN.fit(list(X_train), y_train)

KNeighborsClassifier()

In [137]:
%%time
predicted = KNN.predict(list(X_test))
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.49      0.55      0.52       521
           1       0.54      0.54      0.54      1049
           2       0.50      0.36      0.42        14
           3       0.23      0.31      0.26       652
           4       0.47      0.78      0.58        18
           5       0.51      0.69      0.59        67
           6       1.00      0.20      0.33         5
           7       0.57      0.64      0.61      1136
           8       0.71      0.72      0.71        90
           9       0.56      0.71      0.63       867
          10       0.27      0.43      0.33         7
          11       0.72      0.62      0.67      1564
          12       0.48      0.26      0.34       106
          13       0.53      0.63      0.57      2265
          14       0.65      0.50      0.57       152
          15       0.00      0.00      0.00         5
          16       0.50      0.73      0.59       887
          17       1.00    

In [144]:
y_test

array([ 8, 13, 28, ...,  1, 21, 18])

In [145]:
predicted

array([20, 13, 28, ...,  1, 21, 18])

In [153]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predicted, labels=list(set(labels)))

array([[285,  14,   0, ...,   0,   0,   1],
       [ 24, 570,   0, ...,   1,   0,  11],
       [  0,   0,   5, ...,   0,   0,   0],
       ...,
       [  3,  12,   0, ..., 165,   0,   9],
       [  2,   5,   0, ...,   0,  43,   3],
       [  4,  10,   0, ...,   6,   3, 191]])